In [59]:
import pandas as pd
import numpy as np
import os

In [60]:
df_ckplus_dataset = pd.read_csv("dataset/ckplus_dataset.csv")

In [61]:
def get_session(subject_id, session_id):
    return df_ckplus_dataset[(df_ckplus_dataset["subject_id"] == subject_id) & (df_ckplus_dataset["session_id"] == session_id)]

In [62]:
def read_emotion_features(emotion):
    distance_features = []
    angle_features = []
    with open(f"features/{emotion}.txt", "r") as file:
        for line in file:
            feature = line.strip().split()
            feature = [int(value) for value in feature[0][2:-1].split(",")]
            if len(feature) == 2:
                distance_features.append(feature)
            else:
                angle_features.append(feature)
    return distance_features, angle_features


def load_emotion_features():
    emotions = ["anger", "disgust", "fear", "happiness", "sadness", "surprise"]
    emotion_features = {}
    for emotion in emotions:
        distance_features, angle_features = read_emotion_features(emotion)
        emotion_features[emotion] = (distance_features, angle_features)
    return emotion_features


emotion_features = load_emotion_features()

In [63]:
def euclidean_distance(a, b):
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def smaller_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.arccos(cosine) * 180 / np.pi


def get_face_width(data):
    point1 = data[f"landmark_2"].split(" ")
    point1 = [float(point1[0]), float(point1[1])]
    point2 = data[f"landmark_16"].split(" ")
    point2 = [float(point2[0]), float(point2[1])]
    return euclidean_distance(point1, point2)


def generate_distance_features(data, distance_features_template):
    distance_features = []
    for feature in distance_features_template:
        point1 = data[f"landmark_{feature[0]}"].split(" ")
        point1 = [float(point1[0]), float(point1[1])]
        point2 = data[f"landmark_{feature[1]}"].split(" ")
        point2 = [float(point2[0]), float(point2[1])]

        distance = euclidean_distance(point1, point2)
        distance_features.append(distance)

    distance_features = np.array(distance_features)
    distance_features = distance_features / get_face_width(data)

    return distance_features


def generate_angle_features(data, angle_features_template):
    angle_features = []
    for feature in angle_features_template:
        point1 = data[f"landmark_{feature[0]}"].split(" ")
        point1 = [float(point1[0]), float(point1[1])]
        point2 = data[f"landmark_{feature[1]}"].split(" ")
        point2 = [float(point2[0]), float(point2[1])]
        point3 = data[f"landmark_{feature[2]}"].split(" ")
        point3 = [float(point3[0]), float(point3[1])]

        angle = smaller_angle(point1, point2, point3)
        angle_features.append(angle)

    angle_features = np.array(angle_features)
    angle_features = angle_features / 180

    return angle_features

In [64]:
def generate_features(data, feature_emotion, feature_type):
    distance_features_template, angle_features_template = emotion_features[feature_emotion]

    features = np.array([])

    if feature_type == "distance" or feature_type == "both":
        distance_features = generate_distance_features(data, distance_features_template)
        features = np.concatenate((features, distance_features))

    if feature_type == "angle" or feature_type == "both":
        angle_features = generate_angle_features(data, angle_features_template)
        features = np.concatenate((features, angle_features))

    return features


def get_features_from_df(df, feature_emotion, feature_type):
    features = []
    for index, row in df.iterrows():
        features.append(generate_features(row, feature_emotion, feature_type))
    return np.array(features)

In [65]:
features = get_features_from_df(df_ckplus_dataset[:5], "anger", "both")

In [66]:
features

array([[0.04805285, 0.05564052, 0.17266904, 0.17603197, 0.14024903,
        0.13845739, 0.23161302, 0.17919203, 0.32334929, 0.72361381,
        0.72102522, 0.20333555, 0.20094551, 0.22619378, 0.28310719,
        0.24821964, 0.79697194, 0.81040163, 0.99680758, 0.99310617,
        0.77325909, 0.74225232, 0.60151543],
       [0.04726709, 0.0547055 , 0.1720145 , 0.17510849, 0.13910657,
        0.13778077, 0.23288251, 0.17711922, 0.32250337, 0.72256593,
        0.71937901, 0.20547048, 0.19661544, 0.22330049, 0.27886078,
        0.24264232, 0.79338171, 0.80616933, 0.99416923, 0.99961729,
        0.77392786, 0.7433632 , 0.61615058],
       [0.045303  , 0.0522546 , 0.17023354, 0.17280074, 0.13690819,
        0.13646913, 0.23562555, 0.17308128, 0.32065898, 0.72000764,
        0.71558198, 0.20973325, 0.18678667, 0.21579811, 0.2674246 ,
        0.22948863, 0.78641624, 0.79765907, 0.98691351, 0.98492734,
        0.77524929, 0.74530936, 0.64815029],
       [0.04037397, 0.04570236, 0.16523569, 0.166